In [ ]:
import h5py
path = "/Volumes/HEIU/data/bulkice_train.hdf5"
print(path)
f = h5py.File(path, "r")


In [ ]:
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
plt.imshow(make_grid(afm.transpose(1,0)).permute(1,2,0))

In [ ]:
f['T180_13015'].afm

In [ ]:
from dataset.dataset import z_sampler
z_sampler(10, 4, True)

In [ ]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
import time
from dataset import transform as tf

pics = torch.ones(1, 8, 100, 100) * 0.9
pics[..., 50:, 0:50] = 0.1
pics[..., :50, 50:] = 0.1

transform = nn.Sequential(
    tf.PixelShift(),
    tf.Cutout(),
    tf.ColorJitter(),
    tf.Noisy(),
    tf.Blur(),
)
t = time.time()
for i in range(1):
    pics = transform(pics)

print(time.time() - t)


T = torchvision.utils.make_grid(pics.transpose(1,0),pad_value = 0.5)
plt.imshow(T.permute(1,2,0))


In [ ]:
import torch
torch.arange(10)

In [ ]:
a = torch.ones(100, 100)
a[10:20, 10:20]

In [ ]:
import torch
a = torch.rand(100, 100, 3)
b = torch.rand(100, 100) > 0.5
a[b]

In [ ]:
import torch
a = torch.ones((50, 50, 5))
c = torch.rand((50, 50, 5, 3))
mask = a.nonzero()
print(mask.T.shape)
a[tuple(mask.T)]
(c[tuple(mask.T)] + mask) / (50, 50, 5)

In [1]:
import torch
from utils import poscar
box_cls = torch.randint(0, 3, (3, 50, 50), dtype=torch.long)
box_off = torch.rand((3, 50, 50, 3))
print(poscar.boxncls2pos_torch(box_cls, box_off))
print(poscar.boxncls2pos_np(box_cls.numpy(), box_off.numpy()))

(tensor([2, 2, 1,  ..., 2, 2, 1]), tensor([[5.1622e-04, 1.2156e-02, 9.7890e-03],
        [2.3559e-02, 1.2152e-02, 4.3341e-02],
        [2.6065e-01, 6.0238e-03, 8.1952e-02],
        ...,
        [9.2123e-01, 9.8095e-01, 9.4316e-01],
        [7.6736e-01, 9.8972e-01, 9.7548e-01],
        [8.1795e-01, 9.8637e-01, 9.8140e-01]]))
(array([2, 2, 1, ..., 2, 2, 1]), array([[5.16215960e-04, 1.21561003e-02, 9.78896618e-03],
       [2.35593120e-02, 1.21522141e-02, 4.33411908e-02],
       [2.60650357e-01, 6.02380872e-03, 8.19520152e-02],
       ...,
       [9.21230634e-01, 9.80952162e-01, 9.43162503e-01],
       [7.67357290e-01, 9.89719975e-01, 9.75477711e-01],
       [8.17945004e-01, 9.86365290e-01, 9.81396395e-01]]))


In [1]:
import torch
from utils.metrics import Analyser
from utils import poscar
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]
pred_types, pred_pos = poscar.targ2pred(label_type, label_pos)
a = torch.jit.script(Analyser())
pred_types = pred_types.unsqueeze(0).repeat((10, 1, 1, 1, 1))
pred_pos  = pred_pos.unsqueeze(0).repeat((10, 1, 1, 1, 1))
label_type = label_type.unsqueeze(0).repeat((10, 1, 1, 1))
label_pos = label_pos.unsqueeze(0).repeat((10, 1, 1, 1, 1))

In [3]:
import time
t = time.time()
for i in range(1000):
    a(pred_types, pred_pos, label_type, label_pos)
print(time.time() - t)

5.161307096481323


In [49]:
a = torch.rand((100, 100, 3))
mask = torch.nonzero(a, as_tuple=True)
a[mask[0], mask[1], mask[2]]

tensor([True, True, True,  ..., True, True, True])

In [14]:
import time
import torch
from utils.metrics import ConfusionMatrixCounter
cm = torch.randint(0, 100, (1, 2, 1, 3)).numpy()
t = time.time()
ConfusionMatrixCounter._count(cm)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [1]:
import torch
from model import unet_onehot
m = unet_onehot()
img = torch.rand((1, 1, 10, 100, 100))
m(img)

(tensor([[[[[-1.0713e-01, -1.0817e-01, -1.0439e-01,  ..., -1.1020e-01,
             -1.0455e-01, -1.0650e-01],
            [-1.1229e-01, -1.0428e-01, -1.0566e-01,  ..., -1.0471e-01,
             -1.0658e-01, -1.0664e-01],
            [-1.0990e-01, -1.1120e-01, -1.0782e-01,  ..., -1.0499e-01,
             -1.0398e-01, -1.0404e-01],
            ...,
            [-1.1141e-01, -1.0650e-01, -1.0732e-01,  ..., -1.0715e-01,
             -1.0096e-01, -1.0890e-01],
            [-1.0923e-01, -1.0196e-01, -1.0367e-01,  ..., -1.0543e-01,
             -1.0267e-01, -1.0916e-01],
            [-1.1384e-01, -1.0447e-01, -1.1027e-01,  ..., -1.1186e-01,
             -1.0452e-01, -1.0681e-01]],
 
           [[-1.1533e-01, -1.1010e-01, -1.1001e-01,  ..., -1.1361e-01,
             -1.0778e-01, -1.0806e-01],
            [-1.1518e-01, -1.0786e-01, -1.0786e-01,  ..., -1.1135e-01,
             -1.0730e-01, -1.0660e-01],
            [-1.1223e-01, -1.0231e-01, -1.1157e-01,  ..., -1.0742e-01,
             -1.0920e